In [1]:
import numpy as np
import scipy as scp

In [2]:
class Processor:
    
    
    def __init__(self):
        self.N = self.n_signal = 0
        self.alpha = 1
        self.Y = self.y0 = None
        
    def fit(self, Y_fit, y0_fit):
        if self.Y is not None or self.y is not None:
            throw("Error: fit already fitted Processor. Use add_fit().")
        self.Y = Y_fit.copy()
        self.y0 = y0_fit.copy()
        self.n_signal = Y.shape[-1]
        self.N = y.size
        
    def params_init(self, Y, y0):
        n = self.n_signal
        N = self.N
        self.P = np.zeros((n, n-1))
        self.P[np.arange(n-1), np.arange(n-1)] = 1
        self.P[n-1, :-1] = -1
        #self.P[-1, -1] = -1

        self.p = n.zeros((n, 1))
        self.p[n-1, 0] = 1
        
        self.At = np.zeros((N, n+1, n+1))
        self.At[:,-1,-1] = 1
        self.At[:,np.arange(n),np.arange(n)] = 1+Y/(1+y0)[:, np.newaxis]
        #np.diagonal(self.A, axis1=1, axis2=2) = 
        
        self.U = np.diag(np.ones((n,n))*self.alpha)
        
        
    def add_fit(self, Y_add, y0_add):
        if self.n_signal != Y_add.shape[-1]:
            throw("Error: not appropriate added signals length.")
        self.N += Y_add.shape[0]
        self.Y = np.concatenate((self.Y, Y_add))
        self.y0 = np.concatenate((self.y0, y0_add))
    
    def CalcRegr_prepare(equal_const=True):
         n = self.n_signal
        s_len = self.sig_len
        # transform due to equality constraints
        Y = self.Y.copy()
        y0 = self.y0.copy()
        U = self.U
        At = self.At
        P = self.P
        p = self.p
        alpha = self.alpha
        
        if equal_constr:
            y0 = y0 - self.Y[:, -1]
            Y = self.Y[:, :-1] - self.Y[:, -1].reshape(-1, 1)
            self.a0 = a0 = (y0/np.sum(Y*Y, axis=1))[:,np.newaxis]*Y
            a0[np.sum(Y*Y, axis=1) == 0, :] = 0
            self.Q0 = Q0 = Y[:, np.newaxis, :]*Y[:, :, np.newaxis]
            
            # not needed self.b0 = b0 = np.zeros((,)) # needed???
            Tmp = np.dot(At[1:,...].transpose(0,2,1), U).dot(np.eyes((n+1, n+1))[np.newaxis,...] - At[1:,...])
            a0 += 
            
            Tmp = np.dot(P.T, U)
            A_t = np.transpose(np.dot((np.linalg.inv(Tmp.dot(P))).dot(Tmp), At), axes=(1, 0, 2)).dot(P)
            U = Tmp.dot(P)
            
            
        
    def CalcRegression(equal_const=True):
        n = self.n_signal
        s_len = self.sig_len
        # transform due to equality constraints
        

        
    def ProcessTo(self):
        pass
    
    def ProcessFrom(self):
        pass
    
    def Merge(self):
        pass

SyntaxError: can't assign to function call (<ipython-input-2-1171df76abb3>, line 28)

In [ ]:
Wt = np.zeros((100, 10))

In [129]:
np.sum(X*X, axis=1).shape

(100,)

In [131]:
X = np.arange(100*10).reshape(100, 10)
y = np.arange(100)
a0 = (y/np.sum(X*X, axis=1))[:, np.newaxis]*X
a0[np.sum(X*X, axis=1) == 0, :] = 0
Q0 = X[:, np.newaxis, :]*X[:, :, np.newaxis]
b0 = np.zeros((100,)) # needed???

In [6]:
arr = np.arange(2*2*2).reshape(2, 2, 2)
diag_view = np.diagonal(arr, axis1=1, axis2=2)
arr[:,np.arange(2),np.arange(2)] = 0
arr

array([[[0, 1],
        [2, 0]],

       [[0, 5],
        [6, 0]]])

In [4]:
arr = np.arange(2*2*2).reshape(2, 2, 2)

'1.10.2'

In [32]:
tmp.transpose(0, 2, 1)

array([[[ 11.,   6.,   5.,  13.,   5.],
        [  0.,   6.,   2.,   0.,  13.],
        [  1.,   7.,   0.,   8.,   9.],
        [ 11.,   8.,   1.,  17.,  11.],
        [ 11.,   7.,  13.,  16.,   8.]],

       [[  0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.]],

       [[  0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.]]])

In [24]:
test = np.random.randint(0,20,size=(5, 5))

In [25]:
arr.dot(test)

array([[  75,   62,   67,  105,  113],
       [ 275,  167,  192,  345,  388],
       [ 475,  272,  317,  585,  663],
       [ 675,  377,  442,  825,  938],
       [ 875,  482,  567, 1065, 1213]])

In [26]:
arr = np.arange(5*5).reshape(5, 5)
tmp = np.zeros((3,5,5))
tmp[0,:,:]=test
np.transpose(np.dot(arr, tmp), axes=(1, 0, 2))

array([[[   75.,    62.,    67.,   105.,   113.],
        [  275.,   167.,   192.,   345.,   388.],
        [  475.,   272.,   317.,   585.,   663.],
        [  675.,   377.,   442.,   825.,   938.],
        [  875.,   482.,   567.,  1065.,  1213.]],

       [[    0.,     0.,     0.,     0.,     0.],
        [    0.,     0.,     0.,     0.,     0.],
        [    0.,     0.,     0.,     0.,     0.],
        [    0.,     0.,     0.,     0.,     0.],
        [    0.,     0.,     0.,     0.,     0.]],

       [[    0.,     0.,     0.,     0.,     0.],
        [    0.,     0.,     0.,     0.,     0.],
        [    0.,     0.,     0.,     0.,     0.],
        [    0.,     0.,     0.,     0.,     0.],
        [    0.,     0.,     0.,     0.,     0.]]])

In [14]:
np.dot(arr.T, arr).dot(arr)

array([[45000, 49250, 53500, 57750, 62000],
       [48250, 52800, 57350, 61900, 66450],
       [51500, 56350, 61200, 66050, 70900],
       [54750, 59900, 65050, 70200, 75350],
       [58000, 63450, 68900, 74350, 79800]])

In [ ]:
np.invert()